In [ ]:
! pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 3.6 MB/s eta 0:00:00


In [ ]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 16.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
#https://github.com/google-research-datasets/paws
#PAWS Dataset

from datasets import load_dataset
dataset = load_dataset('paws','labeled_final')

Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Dataset paws downloaded and prepared to /root/.cache/huggingface/datasets/paws/labeled_final/1.1.0/8d567c6472623f42bd2cc635cad06932d0f0cd2f897db56013c1180f4317d338. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
#Checkout this paraphrase dataset too!. Scientific text paraphrases
#https://huggingface.co/datasets/HHousen/ParaSCI
from datasets import load_dataset

dataset = load_dataset("HHousen/ParaSCI")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/HHousen___csv/HHousen--ParaSCI-689c3a2f80aab3b8/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
df_train = dataset['train'].to_pandas()

In [ ]:
df_train=df_train[df_train['label']==1] #Label 1 has paraphrases with same semantic meaning. This step is for PAWS dataset

KeyError: ignored

In [ ]:
df_train['combined']='<s>'+df_train.sentence1+'</s>'+'>>'+'<p>'+df_train.sentence2+'</p>'

In [ ]:
df_train.head()

,sentence1,sentence2,combined
0,"for all methods , the tweets were tokenized wi...",the tweets were tokenized and part-ofspeech ta...,"<s>for all methods , the tweets were tokenized..."
1,it was shown by nederhof et al that prefix pro...,"nederhof et al , for instance , show that pref...",<s>it was shown by nederhof et al that prefix ...
2,"first , kikuchi et al proposed a new long shor...","first , kikuchi et al tried to control the len...","<s>first , kikuchi et al proposed a new long s..."
3,with word confusion networks further improves ...,the complexity is dominated by the word confus...,<s>with word confusion networks further improv...
4,fofe can model the word order in a sequence ba...,fofe can model the word order in a sequence us...,<s>fofe can model the word order in a sequence...


In [ ]:
print("Longest length is:\n",df_train.combined.str.len().max())

Longest length is:
 842


In [ ]:
len(df_train.combined)

338717

In [ ]:
df_train['combined'].sample(n=25000).to_csv('combined.txt', sep='\n', index=False, header=False)

In [ ]:
df_train['combined'].sample(n=25000)

272559    <s>singh et al propose to improve the localiza...
128813    <s>multi-user multiple-input multiple-output c...
74021     <s>it was recently experimentally discovered t...
282845    <s>we used the generalized gradient approximat...
109278    <s>visual recognition has made significant pro...
                                ...                        
210943    <s>the pbe form of the generalized gradient ap...
269655    <s>a striking feature of this figure is the ap...
94368     <s>algorithmic random sequences are incomputab...
112622    <s>the most familiar one is the hydrogen atom ...
136176    <s>in recent years , convolutional neural netw...
Name: combined, Length: 25000, dtype: object

## Training the GPT Model

In [ ]:
from transformers import (
    AutoModelWithLMHead,
    AutoConfig,
    Trainer,
    AutoTokenizer,
    TextDataset,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    LineByLineTextDataset
)

import os

In [ ]:
! git lfs install

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.


In [ ]:
! git clone https://huggingface.co/EleutherAI/gpt-neo-125M

Cloning into 'gpt-neo-125M'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 50 (delta 2), reused 0 (delta 0), pack-reused 44
Unpacking objects: 100% (50/50), done.
Filtering content: 100% (3/3), 1.44 GiB | 67.73 MiB/s, done.


In [ ]:
! ls

combined.txt  gpt-neo-125M  sample_data


In [ ]:
def modelTrainer(text_path, epochs, model='./gptneo125M', batch_size=8, cache_dir='cache'):
  print("model and tokenizer to be read!")
  model = AutoModelWithLMHead.from_pretrained(model)
  tokenizer = AutoTokenizer.from_pretrained(model)
  print("model and tokenizer read successfully!")
  data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
  train_dataset=TextDataset(tokenizer=tokenizer, 
                            file_path=text_path, 
                            block_size=256)
  training_args = TrainingArguments(
      output_dir='gpt_neo125M_finetune/{}'.format(os.path.basename(text_path)),
      num_train_epochs=epochs,
      per_device_train_batch_size=batch_size,
      warmup_steps=500,
      save_steps=2000,
      logging_steps=10
  )

  trainer=Trainer(
      model=model,
      args=training_args,
      data_collator=data_collator,
      train_dataset=train_dataset,
      prediction_loss_only=True
  )
  trainer.train()
  trainer.save_model()
  

In [ ]:
 model = AutoModelWithLMHead.from_pretrained('./gpt-neo-125M')
tokenizer = AutoTokenizer.from_pretrained('./gpt-neo-125M')


/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

Embedding(50258, 768)

In [ ]:
data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
train_dataset=LineByLineTextDataset(tokenizer=tokenizer, 
                            file_path='./combined.txt', 
                            block_size=1024)

/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
len(train_dataset)

25000

In [ ]:
training_args = TrainingArguments(
      output_dir='./model/gpt_neo125M_finetune/',
      num_train_epochs=10,
      per_device_train_batch_size=8,
      warmup_steps=500,
      save_steps=2000,
      logging_steps=10,
      prediction_loss_only=True
  )

In [ ]:
trainer=Trainer(
      model=model,
      args=training_args,
      data_collator=data_collator,
      train_dataset=train_dataset    
  )

In [25]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25000
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 31250
  Number of trainable parameters = 125199360
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,4.001300
20,3.827600
30,3.582300
40,3.395700
50,3.148700
60,2.995500
70,2.688800
80,2.550300
90,2.544100
100,2.503700


Saving model checkpoint to ./model/gpt_neo125M_finetune/checkpoint-2000
Configuration saved in ./model/gpt_neo125M_finetune/checkpoint-2000/config.json
Model weights saved in ./model/gpt_neo125M_finetune/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./model/gpt_neo125M_finetune/checkpoint-4000
Configuration saved in ./model/gpt_neo125M_finetune/checkpoint-4000/config.json
Model weights saved in ./model/gpt_neo125M_finetune/checkpoint-4000/pytorch_model.bin
Saving model checkpoint to ./model/gpt_neo125M_finetune/checkpoint-6000
Configuration saved in ./model/gpt_neo125M_finetune/checkpoint-6000/config.json
Model weights saved in ./model/gpt_neo125M_finetune/checkpoint-6000/pytorch_model.bin
Saving model checkpoint to ./model/gpt_neo125M_finetune/checkpoint-8000
Configuration saved in ./model/gpt_neo125M_finetune/checkpoint-8000/config.json
Model weights saved in ./model/gpt_neo125M_finetune/checkpoint-8000/pytorch_model.bin
Saving model checkpoint to ./model/gpt_neo125M_f

Step,Training Loss
10,4.001300
20,3.827600
30,3.582300
40,3.395700
50,3.148700
60,2.995500
70,2.688800
80,2.550300
90,2.544100
100,2.503700


Saving model checkpoint to ./model/gpt_neo125M_finetune/checkpoint-24000
Configuration saved in ./model/gpt_neo125M_finetune/checkpoint-24000/config.json
Model weights saved in ./model/gpt_neo125M_finetune/checkpoint-24000/pytorch_model.bin
Saving model checkpoint to ./model/gpt_neo125M_finetune/checkpoint-26000
Configuration saved in ./model/gpt_neo125M_finetune/checkpoint-26000/config.json
Model weights saved in ./model/gpt_neo125M_finetune/checkpoint-26000/pytorch_model.bin
Saving model checkpoint to ./model/gpt_neo125M_finetune/checkpoint-28000
Configuration saved in ./model/gpt_neo125M_finetune/checkpoint-28000/config.json
Model weights saved in ./model/gpt_neo125M_finetune/checkpoint-28000/pytorch_model.bin
Saving model checkpoint to ./model/gpt_neo125M_finetune/checkpoint-30000
Configuration saved in ./model/gpt_neo125M_finetune/checkpoint-30000/config.json
Model weights saved in ./model/gpt_neo125M_finetune/checkpoint-30000/pytorch_model.bin


Training completed. Do not forget 

TrainOutput(global_step=31250, training_loss=1.2310977950744628, metrics={'train_runtime': 7069.4294, 'train_samples_per_second': 35.364, 'train_steps_per_second': 4.42, 'total_flos': 1.0464172631728128e+16, 'train_loss': 1.2310977950744628, 'epoch': 10.0})

In [26]:
trainer.save_model()

Saving model checkpoint to ./model/gpt_neo125M_finetune/
Configuration saved in ./model/gpt_neo125M_finetune/config.json
Model weights saved in ./model/gpt_neo125M_finetune/pytorch_model.bin


## Test

In [27]:
from transformers import pipeline, AutoModelWithLMHead, AutoTokenizer

In [28]:
model = AutoModelWithLMHead.from_pretrained("/content/model/gpt_neo125M_finetune")

/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
loading configuration file /content/model/gpt_neo125M_finetune/config.json
Model config GPTNeoConfig {
  "_name_or_path": "/content/model/gpt_neo125M_finetune",
  "activation_function": "gelu_new",
  "architectures": [
    "GPTNeoForCausalLM"
  ],
  "attention_dropout": 0,
  "attention_layers": [
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local"
  ],
  "attention_types": [
    [
      [
        "global",
        "local"
      ],
      6
    ]
  ],
  "bos_token_id": 50256,
  "embed_dropout": 0,
  "eos_token_id"

In [29]:
tokenizer = AutoTokenizer.from_pretrained("/content/gpt-neo-125M")

loading configuration file /content/gpt-neo-125M/config.json
Model config GPTNeoConfig {
  "_name_or_path": "/content/gpt-neo-125M",
  "activation_function": "gelu_new",
  "architectures": [
    "GPTNeoForCausalLM"
  ],
  "attention_dropout": 0,
  "attention_layers": [
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local",
    "global",
    "local"
  ],
  "attention_types": [
    [
      [
        "global",
        "local"
      ],
      6
    ]
  ],
  "bos_token_id": 50256,
  "embed_dropout": 0,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": null,
  "layer_norm_epsilon": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neo",
  "num_heads": 12,
  "num_layers": 12,
  "resid_dropout": 0,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index

In [30]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [31]:
def clean_paraphrased(input_sentence):
  p = generator('<s>'+input_sentence+'</s>>><p>', max_length=len(input_sentence)*2+10)
  #print(p)
  return p[0]['generated_text'].split('</s>>><p>')[1].split('</p>')[0]

In [ ]:
#A significant amount of total R&D spend on late stage clinical trials continures to be related to hematology/oncology and chronic conditions.
#As a condition to obtain certain regulatory approvals for the closing of transaction, the company was required to divest certain 

In [32]:
clean_paraphrased("A significant amount of total R&D spend on late stage clinical trials continues to be related to hematology/oncology and chronic conditions.")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'a significant amount of total research time has been devoted to the clinical trial process, which has been attributed with one or more of its primary components.'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')